# LangSmith Tutorial: Using LangSmith without LangChain

This tutorial demonstrates how to use LangSmith for tracing and monitoring your applications **without** using LangChain. LangSmith is a powerful observability platform that can trace any Python function or application.

## What You'll Learn

- How to set up LangSmith tracing in standalone Python applications
- Using the `@traceable` decorator to monitor function calls
- Best practices for tracing custom functions
- How to organize traces with custom names and projects

## Prerequisites

- Python 3.8+
- LangSmith account and API key
- `langsmith` package installed (`pip install langsmith`)

## Environment Setup

First, let's set up the necessary environment variables for LangSmith:

In [ ]:
import os

# Set up LangSmith environment variables
# Make sure to set your actual API key in your environment
api_key = os.getenv("LANGCHAIN_API_KEY")
if not api_key:
    print("⚠️  Warning: LANGCHAIN_API_KEY not found in environment variables")
    print("   Please set your LangSmith API key:")
    print("   export LANGCHAIN_API_KEY='your_api_key_here'")
else:
    print("✅ LangSmith API key found")

# Enable tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# Set a custom project name for better organization
os.environ["LANGCHAIN_PROJECT"] = "LangSmith-Without-LangChain-Tutorial"

print("🔍 LangSmith tracing enabled")
print(f"📊 Project: {os.environ.get('LANGCHAIN_PROJECT')}")

## Best Practices Summary

1. **Use descriptive names**: Give your traced functions clear, descriptive names using the `name` parameter
2. **Add relevant metadata**: Include context that helps with debugging and analysis
3. **Use tags for organization**: Tag functions by category, team, or feature for better filtering
4. **Handle errors gracefully**: Let LangSmith capture errors while maintaining proper error handling
5. **Organize by projects**: Use different project names for different applications or environments
6. **Don't over-trace**: Focus on key functions and decision points rather than every small utility function

## What's Next?

- Visit the LangSmith dashboard to see your traces
- Set up alerts for errors or performance issues
- Use the LangSmith SDK for more advanced features
- Integrate with CI/CD pipelines for automated monitoring

## Resources

- [LangSmith Documentation](https://docs.smith.langchain.com/)
- [LangSmith Python SDK](https://github.com/langchain-ai/langsmith-sdk)
- [LangSmith API Reference](https://api.smith.langchain.com/docs)

---

**🎉 Congratulations!** You've learned how to use LangSmith for tracing Python applications without LangChain. Your traces should now be visible in the LangSmith dashboard under the "LangSmith-Without-LangChain-Tutorial" project.

In [ ]:
import json
from datetime import datetime

# Mock API client for demonstration
class MockAPIClient:
    def __init__(self):
        self.base_url = "https://api.example.com"
    
    def get_user_data(self, user_id: str) -> dict:
        # Simulate API call
        time.sleep(0.1)
        return {
            "id": user_id,
            "name": f"User {user_id}",
            "email": f"user{user_id}@example.com",
            "created_at": datetime.now().isoformat()
        }

@traceable(name="API Request Handler")
def fetch_user_data(user_id: str) -> dict:
    """Fetch user data from API with tracing."""
    client = MockAPIClient()
    
    # Add metadata about the request
    from langsmith import get_current_run_tree
    run = get_current_run_tree()
    if run:
        run.add_metadata({
            "api_endpoint": f"{client.base_url}/users/{user_id}",
            "request_time": datetime.now().isoformat()
        })
        run.add_tags(["api-call", "user-data"])
    
    try:
        user_data = client.get_user_data(user_id)
        return user_data
    except Exception as e:
        if run:
            run.add_metadata({"error": str(e)})
        raise

@traceable(name="Data Processor")
def process_user_data(user_data: dict) -> dict:
    """Process user data with additional information."""
    processed = {
        "user_info": user_data,
        "processed_at": datetime.now().isoformat(),
        "display_name": user_data.get("name", "Unknown User"),
        "account_age_days": 30  # Mock calculation
    }
    return processed

@traceable(name="User Data Pipeline")
def user_data_pipeline(user_id: str) -> dict:
    """Complete pipeline for fetching and processing user data."""
    
    # Fetch data
    raw_data = fetch_user_data(user_id)
    
    # Process data
    processed_data = process_user_data(raw_data)
    
    # Add pipeline metadata
    from langsmith import get_current_run_tree
    run = get_current_run_tree()
    if run:
        run.add_metadata({
            "pipeline_version": "1.0",
            "user_id": user_id,
            "steps_completed": ["fetch", "process"]
        })
    
    return processed_data

# Test the pipeline
print("=== User Data Pipeline Test ===")
result = user_data_pipeline("12345")
print(json.dumps(result, indent=2))

## Real-World Example: API Integration

Here's a practical example of how you might use LangSmith to trace an application that makes API calls:

In [ ]:
@traceable(name="Error-Prone Function")
def risky_operation(data: str, should_fail: bool = False) -> str:
    """A function that might fail to demonstrate error tracing."""
    
    if should_fail:
        raise ValueError(f"Intentional error for demonstration with data: {data}")
    
    if not data:
        raise ValueError("Input data cannot be empty")
    
    return f"Successfully processed: {data}"

# Test successful operation
try:
    result = risky_operation("test data")
    print(f"✅ Success: {result}")
except Exception as e:
    print(f"❌ Error: {e}")

# Test error scenario
try:
    result = risky_operation("test data", should_fail=True)
    print(f"✅ Success: {result}")
except Exception as e:
    print(f"❌ Error: {e}")

# Test with empty data
try:
    result = risky_operation("")
    print(f"✅ Success: {result}")
except Exception as e:
    print(f"❌ Error: {e}")

## Error Handling and Tracing

LangSmith automatically captures errors and exceptions, making debugging easier:

In [ ]:
from langsmith import traceable

@traceable(
    name="Enhanced Data Processor",
    metadata={"version": "1.0", "author": "tutorial"},
    tags=["data-processing", "example"]
)
def enhanced_process_data(data: str, processing_mode: str = "standard") -> dict:
    """Enhanced data processing with metadata and tags."""
    
    # Add runtime metadata
    from langsmith import get_current_run_tree
    run = get_current_run_tree()
    if run:
        run.add_metadata({
            "processing_mode": processing_mode,
            "input_length": len(data)
        })
        run.add_tags(["runtime-tag"])
    
    # Process data based on mode
    if processing_mode == "detailed":
        result = {
            "original": data,
            "processed": data.upper(),
            "length": len(data),
            "words": data.split(),
            "reversed": data[::-1],
            "mode": processing_mode
        }
    else:
        result = {
            "processed": data.upper(),
            "length": len(data),
            "mode": processing_mode
        }
    
    return result

# Test with different modes
print("=== Standard Mode ===")
result1 = enhanced_process_data("Hello LangSmith!")
print(result1)

print("\n=== Detailed Mode ===")
result2 = enhanced_process_data("Hello LangSmith!", processing_mode="detailed")
print(result2)

## Adding Metadata and Tags

You can enhance your traces with additional metadata and tags for better organization and filtering:

In [ ]:
import time
import random

@traceable(name="Text Analyzer")
def analyze_text(text: str) -> dict:
    """Analyze text using various sub-functions."""
    
    # Simulate some processing time
    time.sleep(0.1)
    
    # Call other traced functions
    word_stats = get_word_statistics(text)
    sentiment = analyze_sentiment(text)
    summary = create_summary(text)
    
    return {
        "word_stats": word_stats,
        "sentiment": sentiment,
        "summary": summary,
        "timestamp": time.time()
    }

@traceable(name="Word Statistics")
def get_word_statistics(text: str) -> dict:
    """Get basic word statistics."""
    words = text.split()
    return {
        "word_count": len(words),
        "char_count": len(text),
        "avg_word_length": sum(len(word) for word in words) / len(words) if words else 0
    }

@traceable(name="Sentiment Analysis")
def analyze_sentiment(text: str) -> str:
    """Mock sentiment analysis."""
    time.sleep(0.05)  # Simulate API call
    sentiments = ["positive", "negative", "neutral"]
    return random.choice(sentiments)

@traceable(name="Text Summarizer")
def create_summary(text: str) -> str:
    """Create a simple summary."""
    words = text.split()
    if len(words) <= 10:
        return text
    return " ".join(words[:10]) + "..."

# Test nested tracing
sample_text = "This is a comprehensive example of using LangSmith for tracing Python applications without LangChain. It demonstrates nested function calls and hierarchical tracing."

analysis_result = analyze_text(sample_text)
print("Analysis Result:")
for key, value in analysis_result.items():
    print(f"  {key}: {value}")

## Advanced Usage: Nested Function Calls

LangSmith can trace nested function calls, creating a hierarchical view of your application's execution flow:

In [ ]:
from langsmith import traceable

# Example 1: Basic function tracing
@traceable
def process_text(text: str) -> str:
    """A simple text processing function."""
    return text.upper().strip()

# Example 2: Function with custom name in LangSmith UI
@traceable(name="Data Processor")
def process_data(data: str) -> dict:
    """Process data and return structured information."""
    return {
        "original": data,
        "processed": data.upper(),
        "length": len(data),
        "word_count": len(data.split())
    }

# Test the functions
result1 = process_text("  hello world  ")
print(f"Basic processing result: {result1}")

result2 = process_data("This is a sample text for processing")
print(f"Data processing result: {result2}")

## Basic Usage: The @traceable Decorator

The `@traceable` decorator is the core feature for adding observability to your functions. It automatically captures:

- Function inputs and outputs
- Execution time
- Error information (if any)
- Custom metadata and tags

Let's start with a simple example: